# Email Generation Process

In [1]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_groq import ChatGroq

In [5]:
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.1-70b-versatile" 
)

response = llm.invoke("Tell me the first persident of India ?")
print(response.content)

The first President of India was Dr. Rajendra Prasad. He served as the President of India from January 26, 1950, to May 13, 1962.


In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-41101?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Part Time Athlete (Retail Associate) - Nike, Brisbane Airport DFO

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresPart Time Athlete (Retail Associate) - Nike, Bri

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Part Time Athlete (Retail Associate)',
 'experience': 'Previous relevant retail and/or customer-service experience',
 'skills': ['Delivering premium customer service',
  'Building and maintaining an expert level of product knowledge',
  'Replenishing and maintaining stock',
  'Maintaining visual merchandising standards in store',
  'Assisting the store to achieve Key Performance Indicators',
  'A zeal for delivering a premium service experience',
  'A passion for the NIKE brand/sports',
  'A natural sense of urgency',
  'An energetic demeanor'],
 'description': 'As an Athlete you will be a key player within the team, where dedication and commitment are as fundamental as any previous work history you bring with you. You will be the face of Nike and act as the initial touch-point for our customers. You will have a genuine passion for the Nike brand and you will represent Nike with a positive, athletic spirit.'}

In [9]:
type(json_res)

dict

In [10]:
import pandas as pd

df = pd.read_csv("/Users/hash/Downloads/portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [11]:
import uuid

In [12]:
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfoliio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())]) 

In [15]:
links = collection.query(query_texts=['Expertise in python','Expertise in React Native'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [16]:
job = json_res
job['skills']

['Delivering premium customer service',
 'Building and maintaining an expert level of product knowledge',
 'Replenishing and maintaining stock',
 'Maintaining visual merchandising standards in store',
 'Assisting the store to achieve Key Performance Indicators',
 'A zeal for delivering a premium service experience',
 'A passion for the NIKE brand/sports',
 'A natural sense of urgency',
 'An energetic demeanor']

In [17]:
links = collection.query(query_texts=job['skills'] , n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/react-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}]]

In [18]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Lakshya, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhance Your Retail Experience with AtliQ's Innovative Solutions

Dear Hiring Manager,

I came across the job description for a Part Time Athlete (Retail Associate) at Nike, and I was impressed by the emphasis on delivering premium customer service and maintaining a high level of product knowledge. As a Business Development Executive at AtliQ, I believe our company can help Nike streamline its retail operations and enhance the overall customer experience.

At AtliQ, we specialize in developing customized software solutions that cater to the unique needs of our clients. Our expertise in creating intuitive and user-friendly applications can help Nike's retail associates stay up-to-date with the latest product information, manage inventory efficiently, and provide exceptional customer service.

Our portfolio showcases our capabilities in developing innovative solutions for various industries, including retail. Some of our notable projects include:

* A mobile application built us